c. Solving for your backward Euler iterates will require inverting a system of two variables. It may
prove impossible to do analytically so we will propose that we can solve for it by finding a zero of
the system: 

xn+1 − xn − ∆tfx(xn+1, yn+1) 
yn+1 − yn − ∆tfy(xn+1, yn+1)

are two values in a vector that are equal to the zero vector

where fx(x, y) = dx
dt and fy(x, y) = dy
dt .
Provide pseudocode for a Newton-Raphson solver for this system, assuming (xn, yn) is a good initial
guess as to (xn+1, yn+1). You can assume ten rounds of N-R is enough for the N-R to converge.
3


In [178]:
a, b, c, d, x0, y0, T, delta_t = 1, .25, .1, 1, 50, 1, 100, .1

In [179]:
#the format of these formulas might need changed
def fx(xn, yn, next_x, next_y):
    return next_x - xn - (delta_t * (a*next_x - b*next_x*next_y))

def fy(xn, yn, next_x, next_y):
    return next_y - yn - (delta_t * (c*next_x*next_y - d*next_y))


In [180]:
#derivatives
def dx(xn, next_y):
    return 1 - delta_t * (a - b * next_y)

def dy(yn, next_x):
    return 1 - delta_t * (c*next_x - d)

In [181]:
def forward_euler(a, b, c, d, x0, y0, T, delta_t):
    x_vals, y_vals = [], []
    xn, yn = x0, y0
    i = 0
    while i < T:
        next_x = xn + delta_t*(a*xn - b*xn*yn)
        next_y = yn + delta_t*(c*xn*yn - d*yn)

        x_vals.append(xn)
        y_vals.append(yn)
        
        xn, yn = next_x, next_y
        i += delta_t
    return x_vals, y_vals


In [198]:
def leap_frog(a, b, c, d, x0, y0, T, delta_t):
    x_vals, y_vals = [], []
    xn, yn = x0, y0
    fe_x, fe_y = forward_euler(a, b, c, d, x0, y0, T, delta_t)
    next_x = fe_x[1]
    next_y = fe_y[1]

    x_vals.append(xn)
    y_vals.append(yn)
    x_vals.append(next_x)
    y_vals.append(next_y)
        
    i = 2
    while i < 15:
        next_x = x_vals[i-2] + 2*delta_t*(a*x_vals[i-1] - b*x_vals[i-1]*y_vals[i-1])
        next_y = y_vals[i-2] + 2*delta_t*(c*x_vals[i-1]*y_vals[i-1] - d*y_vals[i-1])

        x_vals.append(next_x)
        y_vals.append(next_y)

        i += 1
    
    return x_vals, y_vals


In [194]:

def newton_raphson_solver(x0, y0, delta_t, N):
    xn, yn = x0, y0
    next_x, next_y = x0,y0
    for i in range(N):
        next_x = xn - fx(xn, yn, next_x, next_y) / dx(xn, next_y)
        next_y = yn - fy(xn, yn, next_x, next_y) / dy(yn, next_x)

        xn, yn = next_x, next_y 
    
    return xn, yn


          

In [195]:
import csv
with open("newton_raphson.csv", 'w') as csvfile:
    csvwriter = csv.writer(csvfile)
    fields = ["X", "Y"]
    csvwriter.writerow(fields)

    row = [[x0, y0]]
    csvwriter.writerows(row)
    x, y = x0, y0
    for i in range(10):
        x, y = newton_raphson_solver(x, y, delta_t, 10)
        row = [[x, y]]
        csvwriter.writerows(row)
    print("done")



done


In [196]:
with open("forward_euler.csv", 'w') as csvfile:
    csvwriter = csv.writer(csvfile)
    fields = ["X", "Y"]
    csvwriter.writerow(fields)
    
    fe_res = forward_euler(a, b, c, d, x0, y0, T, delta_t)
    x_vals = fe_res[0]
    y_vals = fe_res[1]
    print(y_vals)
    i = 0
    while i < 50:
        x, y = x_vals[i], y_vals[i]
        row = [[x, y]]
        csvwriter.writerows(row)
        i+=1

    print("done")

[1, 1.4, 2.0125, 2.9632804687500003, 4.447529836851793, 6.744474863682497, 10.181169657562847, 14.943251012403557, 20.62172635341791, 25.749997688766925, 28.422593140664475, 28.22304538553489, 26.4226801731415, 24.157776871758372, 21.89361135046977, 19.772409469405815, 17.829187779881018, 16.064849058230244, 14.4693175619695, 13.029275687340347, 11.730928456042774, 10.561028625430307, 9.507244765670434, 8.558265478812144, 7.703793626211409, 6.9344947404757455, 6.241927791206435, 5.618471118764626, 5.057249453136791, 4.552064700253687, 4.097331609433217, 3.6880186582637755, 3.3195941039813475, 2.987976961444239, 2.689492582088753, 2.420832477060987, 2.179018025415129, 1.9613677210169598, 1.7654676319988525, 1.5891447700608872, 1.4304430912514379, 1.2876018737929833, 1.1590362413514803, 1.0433196215358465, 0.9391679492160495, 0.8454254424393484, 0.7610517953463617, 0.6851106476202382, 0.61675920373988, 0.5552388877558391, 0.49986693056988074, 0.4500287968780063, 0.40517136812853183, 0.36

In [197]:
with open("leapfrog.csv", 'w') as csvfile:
    csvwriter = csv.writer(csvfile)
    fields = ["X", "Y"]
    csvwriter.writerow(fields)
    
    lf_res = leap_frog(a, b, c, d, x0, y0, T, delta_t)
    x_vals = lf_res[0]
    y_vals = lf_res[1]

    print(lf_res)
    print(x_vals)
    print(y_vals)
    i = 0
    while i < 50:
        x, y = x_vals[i], y_vals[i]
        row = [[x, y]]
        csvwriter.writerows(row)
        i+=1

    print("done")

([50, 53.75, 56.9875, 58.807640625, 58.4843198503955, 54.03328908221058, 45.10327017395625, 32.57044840678981, 21.58276667008835, 13.315958506834441, 8.597860875604718, 4.975538559694302, 3.9092766518677156, 1.6320671417360373, 2.581132540664978], [1, 1.4, 2.225, 3.4909437500000005, 5.632694559841797, 8.952891043179049, 13.517199348358536, 18.44284905759215, 21.84246681093045, 23.502772988995275, 23.401151211471603, 22.84653959560539, 21.10532006662087, 20.27560628161394, 17.71202182611806])
[50, 53.75, 56.9875, 58.807640625, 58.4843198503955, 54.03328908221058, 45.10327017395625, 32.57044840678981, 21.58276667008835, 13.315958506834441, 8.597860875604718, 4.975538559694302, 3.9092766518677156, 1.6320671417360373, 2.581132540664978]
[1, 1.4, 2.225, 3.4909437500000005, 5.632694559841797, 8.952891043179049, 13.517199348358536, 18.44284905759215, 21.84246681093045, 23.502772988995275, 23.401151211471603, 22.84653959560539, 21.10532006662087, 20.27560628161394, 17.71202182611806]


IndexError: list index out of range